In [66]:
import numpy as np
import plotly.express as px
from tools.utils import query_database
import pandas as pd
import psycopg2
import cv2
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns

**Graphs**

- best models
- best accuracy
- best preprocesing
- loss
- accuracy


# Everything


## Top Performing


In [67]:
query = """
with noShot as (
select
    m.name, m.preprocessing, d.shot, d.shuffle, d.optimiser, e.epoch, r.value
from
    model m
    join cnn_data d on m.id = d.model_id
    join epoch e on m.id = e.model_id
    join cnn_result r on e.id = r.epoch_id
where 
    m.type = 'cnn' and d.shot is null and r.type = 'Accuracy' and d.optimiser = 'adam'
order by
    r.value desc
limit 1
),
fiveShot as (
select
    m.name, m.preprocessing, d.shot, d.shuffle, d.optimiser, e.epoch, r.value
from
    model m
    join cnn_data d on m.id = d.model_id
    join epoch e on m.id = e.model_id
    join cnn_result r on e.id = r.epoch_id
where 
    m.type = 'cnn' and d.shot = 5 and r.type = 'Accuracy' and d.optimiser = 'adam'
order by
    r.value desc
limit 1
),
tenShot as (
select
    m.name, m.preprocessing, d.shot, d.shuffle, d.optimiser, e.epoch, r.value
from
    model m
    join cnn_data d on m.id = d.model_id
    join epoch e on m.id = e.model_id
    join cnn_result r on e.id = r.epoch_id
where 
    m.type = 'cnn' and d.shot = 10 and r.type = 'Accuracy' and d.optimiser = 'adam'
order by
    r.value desc
limit 1
),
fifthteenShot as (
select
    m.name, m.preprocessing, d.shot, d.shuffle, d.optimiser, e.epoch, r.value
from
    model m
    join cnn_data d on m.id = d.model_id
    join epoch e on m.id = e.model_id
    join cnn_result r on e.id = r.epoch_id
where 
    m.type = 'cnn' and d.shot = 15 and r.type = 'Accuracy' and d.optimiser = 'adam'
order by
    r.value desc
limit 1
),
combined as (
select * from noShot
union
select * from fiveShot
union
select * from tenShot
union
select * from fifthteenShot
)
select
    concat_ws(', ', name, preprocessing, optimiser, shuffle, epoch) as x_value, value as y_value
from combined
"""

columns, rows = query_database(query)

df = pd.DataFrame(rows, columns=columns)
df["y_value"] = df["y_value"].astype(float)
df["x_value"] = df["x_value"].astype(str)
df

,x_value,y_value
0,"resnet18_transfer, cqt, adam, t, 36",0.589558
1,"resnet18, combined2, adam, 29",0.983839
2,"resnet18_transfer, cqt, adam, t, 40",0.670566
3,"resnet18, cqt, adam, f, 31",0.613323


In [68]:
x = df["x_value"]
y = df["y_value"]

fig = go.Figure(
    data=[
        go.Bar(
            x=x,
            y=y,
            text=y,
            textposition="auto",
        )
    ]
)
fig.update_layout(
    title="Top Performing Models of Each Shot",
    xaxis_title="Shot",
    yaxis_title="Accuracy",
)
fig.show()

In [85]:
query = """
with fiveShot as (
select
    m.name, m.preprocessing, d.k_spt, e.epoch, ms.step, r.update, r.accuracy
from
    model m
    join maml_data d on m.id = d.model_id
    join epoch e on m.id = e.model_id
    join maml_step ms on e.id = ms.epoch_id
    join maml_update_acc r on e.id = r.maml_step_id
where 
    m.type = 'maml' and d.k_spt = 5
order by
    r.accuracy desc
limit 1
),
tenShot as (
select
    m.name, m.preprocessing, d.k_spt, e.epoch, ms.step, r.update, r.accuracy
from
    model m
    join maml_data d on m.id = d.model_id
    join epoch e on m.id = e.model_id
    join maml_step ms on e.id = ms.epoch_id
    join maml_update_acc r on e.id = r.maml_step_id
where 
    m.type = 'maml' and d.k_spt = 10
order by
    r.accuracy desc
limit 1
),
fifthteenShot as (
select
    m.name, m.preprocessing, d.k_spt, e.epoch, ms.step, r.update, r.accuracy
from
    model m
    join maml_data d on m.id = d.model_id
    join epoch e on m.id = e.model_id
    join maml_step ms on e.id = ms.epoch_id
    join maml_update_acc r on e.id = r.maml_step_id
where 
    m.type = 'maml' and d.k_spt = 15
order by
    r.accuracy desc
limit 1
),
combined as (
select * from fiveShot
union
select * from tenShot
union
select * from fifthteenShot
)
select
    concat_ws(', ', name, preprocessing, k_spt, epoch, step, update) as x_value, accuracy as y_value
from combined
"""

columns, rows = query_database(query)

df = pd.DataFrame(rows, columns=columns)
df["y_value"] = df["y_value"].astype(float)
df["x_value"] = df["x_value"].astype(str)
df

,x_value,y_value
0,"vggnet, gammatone, 15, 6, 0, 5",0.606667
1,"vggnet, stft, 10, 8, 0, 4",0.553333
2,"original, stft, 5, 9, 0, 4",0.426667


In [86]:
x = df["x_value"]
y = df["y_value"]

fig = go.Figure(
    data=[
        go.Bar(
            x=x,
            y=y,
            text=y,
            textposition="auto",
        )
    ]
)
fig.update_layout(
    title="Top Performing Models of Each Shot",
    xaxis_title="Shot",
    yaxis_title="Accuracy",
)
fig.show()

# Heatmap


In [69]:
def create_heatmap(x, y, z, title):
    annotations = []
    for i in range(len(z)):
        for j in range(len(z[i])):
            annotations.append(
                dict(
                    text=str(z[i][j]),
                    x=j,
                    y=i,
                    font=dict(color="white" if z[i][j] > np.max(z) / 2 else "black"),
                    showarrow=False,
                )
            )

    heatmap = go.Heatmap(z=z, colorscale="YlGnBu")

    # Define layout
    layout = go.Layout(
        title=title,
        annotations=annotations,
        xaxis=dict(ticks="", side="bottom", tickvals=np.arange(len(z[0])), ticktext=x),
        yaxis=dict(ticks="", ticksuffix=" ", tickvals=np.arange(len(z)), ticktext=y),
    )

    # Create figure
    fig = go.Figure(data=[heatmap], layout=layout)

    return fig

In [70]:
query = """
with cnn_top as (
select
    max(cr.value) as max, cd.shot as shot, cd.shuffle as shuffle
from
    model m
    join epoch e on m.id = e.model_id
    join cnn_result cr on e.id = cr.epoch_id
    join cnn_data cd on m.id = cd.model_id
where
    cr.type = 'Accuracy' and cd.shot is not null
group by
    m.id, cd.shot, cd.shuffle
),
maml_top as (
select
    max(mr.accuracy) as max, md.k_spt as shot
from
    model m
    join epoch e on m.id = e.model_id
    join maml_step ms on e.id = ms.epoch_id
    join maml_update_acc mr on ms.id = mr.maml_step_id
    join maml_data md on m.id = md.model_id
group by
    m.id, md.k_spt
)
select
    max, shot, shuffle
from
    cnn_top
union
select
    max, shot, null as shuffle
from
    maml_top
"""


columns, rows = query_database(query)

df = pd.DataFrame(rows, columns=columns)

df["index"] = df.index
df["max"] = df["max"].astype(float)
df["shot"] = df["shot"].astype(int)

df

,max,shot,shuffle,index
0,0.350684,5,True,0
1,0.262372,10,False,1
2,0.504517,15,False,2
3,0.347334,15,True,3
4,0.370301,10,False,4
...,...,...,...,...
389,0.448489,5,True,389
390,0.382400,15,False,390
391,0.587283,15,False,391
392,0.266923,10,True,392


In [71]:
shot = [5, 10, 15]
shot_counts = {s: {} for s in shot}
entries = [round(i * 0.1, 2) for i in range(11)]

no_shuffle = [[0 for _ in range(10)] for _ in range(3)]
shuffle = [[0 for _ in range(10)] for _ in range(3)]
maml = [[0 for _ in range(10)] for _ in range(3)]

# Initialize a dictionary to hold counts for each interval
interval_counts = {}

# Define the interval width
interval_width = 0.10

# Iterate over the values and count them in intervals
for index, row in df.iterrows():
    interval = (
        int(row["max"] / interval_width) * interval_width
    )  # Determine the interval

    interval = round(interval, 2)
    entries_idx = entries.index(interval)

    if row["shot"] == 5 and row["shuffle"] == False:
        no_shuffle[0][entries_idx] += 1

    elif row["shot"] == 10 and row["shuffle"] == False:
        no_shuffle[1][entries_idx] += 1

    elif row["shot"] == 15 and row["shuffle"] == False:
        no_shuffle[2][entries_idx] += 1

    elif row["shot"] == 5 and row["shuffle"]:
        shuffle[0][entries_idx] += 1

    elif row["shot"] == 10 and row["shuffle"]:
        shuffle[1][entries_idx] += 1

    elif row["shot"] == 15 and row["shuffle"]:
        shuffle[2][entries_idx] += 1

    elif row["shot"] == 5 and row["shuffle"] is None:
        maml[0][entries_idx] += 1

    elif row["shot"] == 10 and row["shuffle"] is None:
        maml[1][entries_idx] += 1

    elif row["shot"] == 15 and row["shuffle"] is None:
        maml[2][entries_idx] += 1

## No Shuffle


In [72]:
fig = create_heatmap(entries, shot, no_shuffle, "No Shuffle Accuracy")

fig.show()

## Shuffle


In [73]:
fig = create_heatmap(entries, shot, shuffle, "Shuffle Accuracy")

fig.show()

## MaML


In [74]:
fig = create_heatmap(entries, shot, maml, "MaML Accuracy")

fig.show()

# STFT


In [115]:
query = """
with ranked_results AS (
    select
        m.id,
        r.value,
        ROW_NUMBER() OVER (ORDER BY r.value DESC) AS rank_high,
        ROW_NUMBER() OVER (ORDER BY r.value ASC) AS rank_low
    from
        model m
        JOIN epoch e ON m.id = e.model_id
        JOIN cnn_result r ON e.id = r.epoch_id
        join cnn_data d on m.id = d.model_id
    where
        r.type = 'Accuracy' and m.type ='cnn' and d.shot is null and m.preprocessing = 'stft'
), best_worst as (
select
    id,
    value
from
    ranked_results
where
    rank_high = 1
union
select
    id,
    value
from
    ranked_results
where
    rank_low = 1
)
select
    bw.value, bw.id, m.name
from
    best_worst bw
    join model m on bw.id = m.id
    join cnn_data d on bw.id = d.model_id
order by
    bw.value desc
"""

columns, rows = query_database(query)

df = pd.DataFrame(rows, columns=columns)
df

,value,id,name
0,0.9755856394767761,914,resnet18
1,0.20000000298023224,915,vggnet16


## CNN


### 5


In [117]:
query = """
with ranked_results AS (
    select
        m.id,
        r.value,
        ROW_NUMBER() OVER (ORDER BY r.value DESC) AS rank_high,
        ROW_NUMBER() OVER (ORDER BY r.value ASC) AS rank_low
    from
        model m
        JOIN epoch e ON m.id = e.model_id
        JOIN cnn_result r ON e.id = r.epoch_id
        join cnn_data d on m.id = d.model_id
    where
        r.type = 'Accuracy' and m.type ='cnn' and d.shot = 5 and m.preprocessing = 'stft'
), best_worst as (
select
    id,
    value
from
    ranked_results
where
    rank_high = 1
union
select
    id,
    value
from
    ranked_results
where
    rank_low = 1
)
select
    bw.value, bw.id, m.name
from
    best_worst bw
    join model m on bw.id = m.id
    join cnn_data d on bw.id = d.model_id
order by
    bw.value desc
"""

columns, rows = query_database(query)

df = pd.DataFrame(rows, columns=columns)
df

,value,id,name
0,0.5751534700393677,682,resnet18_transfer
1,0.16569329798221588,433,resnet18_transfer


### 10


In [118]:
query = """
with ranked_results AS (
    select
        m.id,
        r.value,
        ROW_NUMBER() OVER (ORDER BY r.value DESC) AS rank_high,
        ROW_NUMBER() OVER (ORDER BY r.value ASC) AS rank_low
    from
        model m
        JOIN epoch e ON m.id = e.model_id
        JOIN cnn_result r ON e.id = r.epoch_id
        join cnn_data d on m.id = d.model_id
    where
        r.type = 'Accuracy' and m.type ='cnn' and d.shot = 10 and m.preprocessing = 'stft'
), best_worst as (
select
    id,
    value
from
    ranked_results
where
    rank_high = 1
union
select
    id,
    value
from
    ranked_results
where
    rank_low = 1
)
select
    bw.value, bw.id, m.name
from
    best_worst bw
    join model m on bw.id = m.id
    join cnn_data d on bw.id = d.model_id
order by
    bw.value desc
"""

columns, rows = query_database(query)

df = pd.DataFrame(rows, columns=columns)
df

,value,id,name
0,0.563451886177063,714,resnet18_transfer
1,0.138272225856781,499,vggnet16_transfer


### 15


In [119]:
query = """
with ranked_results AS (
    select
        m.id,
        r.value,
        ROW_NUMBER() OVER (ORDER BY r.value DESC) AS rank_high,
        ROW_NUMBER() OVER (ORDER BY r.value ASC) AS rank_low
    from
        model m
        JOIN epoch e ON m.id = e.model_id
        JOIN cnn_result r ON e.id = r.epoch_id
        join cnn_data d on m.id = d.model_id
    where
        r.type = 'Accuracy' and m.type ='cnn' and d.shot = 15 and m.preprocessing = 'stft'
), best_worst as (
select
    id,
    value
from
    ranked_results
where
    rank_high = 1
union
select
    id,
    value
from
    ranked_results
where
    rank_low = 1
)
select
    bw.value, bw.id, m.name
from
    best_worst bw
    join model m on bw.id = m.id
    join cnn_data d on bw.id = d.model_id
order by
    bw.value desc
"""

columns, rows = query_database(query)

df = pd.DataFrame(rows, columns=columns)
df

,value,id,name
0,0.6013572216033936,842,resnet18_transfer
1,0.17239266633987427,564,vggnet16_transfer
